# Example Jupyter Notebook for CPS

Welcome to the example Jupyter Notebook for CPS.
In this notebook you will be guided through the steps needed to setup the connection to CPS, select your Knowledge Graph and make some example queries.

Navigate all examples notebooks:
- [/examples/notebooks](./)

**⚠️⚠️ Disclaimer ⚠️⚠️**: Notebooks executed on the CPS FoC system are **not persistent**. This means that your session will be completely reset once restarted. Please remember to download your modified notebooks locally to restore your work at a later point.

In [ ]:
import os
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from IPython.display import display, Markdown
import pandas as pd

import deepsearch.cps.ipython
from deepsearch.core.client import DeepSearchBearerTokenAuth, DeepSearchConfig
from deepsearch.cps.client.api import CpsApi, CpsApiClient, DeepSearchConfig
from deepsearch.cps.ipython.kg_widgets import KGSelector

from deepsearch.cps.client.queries import Query

### Initialized CPS integration in Notebooks
This section will authenticate the user to CPS

In [ ]:
config = DeepSearchConfig(
    host=os.getenv("CPS_URL"),
    auth=DeepSearchBearerTokenAuth(bearer_token=os.getenv("CPS_ACCESS_TOKEN")),
    verify_ssl=False,
)

client = CpsApiClient(config)
api = CpsApi(client)

print('Connected to:', client.config.host)

### Select the KG
Use the interactive components to select the KG you want to use

In [ ]:
selector = KGSelector(api)
kg = selector.interactive()

## KG Queries

### Query the KG details

In [ ]:
# Get KG topoogy
topo_resp = kg.get_topology()

topo_nodes = [
    {
        'name': node['name'],
        'size': node['size'],
    }
    for i, node in enumerate(topo_resp['nodes']['categories'])
]

topo_edges = [
    {
        'name': edge['name'],
        'source': edge['categories']['source'],
        'target': edge['categories']['target'],
        'size': edge['matrix']['NNZ'],
    }
    for i, edge in enumerate(topo_resp['edges'])
]


# Print 
display(Markdown('**Nodes**'), pd.json_normalize(topo_nodes))
display(Markdown('**Edges**'), pd.json_normalize(topo_edges))

# Output the visual KG topology
# display_kg_topology(kg)

### Build a query workflow and run

In [ ]:
query = Query()

# build here your query (see examples in SDK documentation)

# For debug: print the whole query
#print(json.dumps(query.to_flow(), indent=2))

In [ ]:
# Execute the query
result = api.queries.run(query)

# Visualize the outputs as table
for name, output in result.outputs.items():
    display(Markdown(f"## Output '{name}'"))
    display(pd.json_normalize(output))